In [ ]:
'''
!pip3 install 'import_ipynb'
!pip3 install 'torch==1.4.0'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
'''


"\n!pip3 install 'import_ipynb'\n!pip3 install 'torch==1.4.0'\n!pip3 install 'torchvision==0.5.0'\n!pip3 install 'Pillow-SIMD'\n"

In [ ]:

import os
import logging

import torch
import copy
import torch.nn as nn
import numpy as np

from torchsummary import summary

if not os.path.isdir('./icarl_vitto'):
  !git clone https://github.com/DavideNapolitano/icarl_vitto

from icarl_vitto.cifar100Class import cifar_100

Cloning into 'icarl_vitto'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 99 (delta 53), reused 89 (delta 43), pack-reused 0
Unpacking objects: 100% (99/99), done.


In [ ]:
"""
Credits to @hshustc
Taken from https://github.com/hshustc/CVPR19_Incremental_Learning/tree/master/cifar100-class-incremental
"""

import torch.nn as nn

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)
            
        out += residual
        out = self.relu(out)
        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=100): #METTERE A 10
        super(ResNet, self).__init__()
        self.inplanes = 16
        self.block=block
        self.out_fc=num_classes
        self.in_fc=64
        self.convolutional=nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1,bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            self._make_layer(block, 16, layers[0]),
            self._make_layer(block, 32, layers[1], stride=2),
            self._make_layer(block, 64, layers[2], stride=2),
        )
        self.avgpool = nn.AvgPool2d(8, stride=1)
        self.classifier=nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x=self.convolutional(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def forward(self, x):
        return self._forward_impl(x)

    def get_conv(self):
        return self.convolutional

    def get_fm_out(self,x):
        x=self.convolutional(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return x
    
    def get_fc(self):
        return self.classifier

    def get_in_fc(self):
        return self.in_fc
    
    def get_out_fc(self):
        return self.out_fc
    
    def increment_classes(self,n=10):
        in_features=self.classifier.in_features
        out_features = self.classifier.out_features
        weight = self.classifier.weight.data
        #bias = self.classifier[0].bias.data

        self.classifier = nn.Linear(in_features, out_features+n, bias=False)
        self.classifier.weight.data[:out_features] = weight
        #self.classifier[0].bias.data[:out_features] = bias
        self.out_fc += n

def resnet32(pretrained=False, **kwargs):
    n = 5
    model = ResNet(BasicBlock, [n, n, n], **kwargs)
    return model

print("IMPORT NET DONE")

IMPORT NET DONE
